In [14]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transformations for dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Load CIFAR-10 dataset from the 'Q1_Data' folder without re-downloading
cifar10_data = datasets.CIFAR10(root='./Q1_Data', train=True, download=False, transform=transform)

# Filter only cats (class 3) and dogs (class 5)
cat_dog_data = [(img, label) for img, label in cifar10_data if label in [3, 5]]

# Create DataLoader for the filtered dataset
train_loader = DataLoader(cat_dog_data, batch_size=64, shuffle=True)


In [16]:
import os
import torch
import torch.nn as nn
import numpy as np
from torchvision.utils import save_image

# Create the directory for fake images if it doesn't exist
if not os.path.exists('./Fake_Images'):
    os.makedirs('./Fake_Images')

# Generator network
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100, 256, 4, 1, 0, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )
    
    def forward(self, x):
        return self.main(x)

# Siamese-like Discriminator network
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.fc = nn.Sequential(
            nn.Linear(256*4*4*2, 1),  # 2 because we concatenate two images (real + generated)
            nn.Sigmoid()
        )
    
    def forward(self, img_real, img_generated):
        features_real = self.cnn(img_real).view(img_real.size(0), -1)
        features_generated = self.cnn(img_generated).view(img_generated.size(0), -1)
        combined_features = torch.cat((features_real, features_generated), 1)
        similarity_score = self.fc(combined_features)
        
        # Print similarity score for real and fake images
        print(f"Similarity Score (Real): {similarity_score[0].item():.4f} (Fake): {similarity_score[1].item():.4f}")
        
        return similarity_score

# Initialize networks
generator = Generator()
discriminator = Discriminator()

# Set the loss criterion
criterion = nn.BCELoss()

# Optimizers for Generator and Discriminator
optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Define number of epochs
epochs = 50

# Check if CUDA is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the models to the selected device
generator.to(device)
discriminator.to(device)

# Training loop
for epoch in range(epochs):
    for i, (imgs, _) in enumerate(train_loader):
        
        # Ground truths for real and fake
        real_labels = torch.ones(imgs.size(0), 1).to(device)
        fake_labels = torch.zeros(imgs.size(0), 1).to(device)
        
        # Move real images to device
        imgs = imgs.to(device)

        ### Train Discriminator ###
        
        # Generate fake images
        noise = torch.randn(imgs.size(0), 100, 1, 1).to(device)
        fake_imgs = generator(noise)
        
        # Get discriminator predictions on real and fake images
        real_score = discriminator(imgs, imgs)  # Similarity score for real
        fake_score = discriminator(imgs, fake_imgs)  # Similarity score for fake
        
        # Calculate the loss for real and fake
        d_loss_real = criterion(real_score, real_labels)
        d_loss_fake = criterion(fake_score, fake_labels)
        
        # Total discriminator loss
        d_loss = d_loss_real + d_loss_fake
        
        # Backpropagation and optimization for Discriminator
        optimizer_D.zero_grad()
        d_loss.backward()
        optimizer_D.step()

        ### Train Generator ###
        
        # Generate more fake images
        noise = torch.randn(imgs.size(0), 100, 1, 1).to(device)
        fake_imgs = generator(noise)
        
        # Get discriminator's similarity score for fake images
        fake_score = discriminator(imgs, fake_imgs)
        
        # Generator loss (wants to minimize the dissimilarity score)
        g_loss = criterion(fake_score, real_labels)  # Generator wants similarity with real images
        
        # Backpropagation and optimization for Generator
        optimizer_G.zero_grad()
        g_loss.backward()
        optimizer_G.step()

        # Save generated fake images every few epochs
        if (i + 1) % 100 == 0:  # Save every 100 batches
            save_image(fake_imgs.data, f'./Fake_Images/fake_images_epoch_{epoch+1}_batch_{i+1}.png', normalize=True)

    print(f"Epoch [{epoch}/{epochs}] | D Loss: {d_loss.item():.4f} | G Loss: {g_loss.item():.4f}")


Similarity Score (Real): 0.6712 (Fake): 0.4605
Similarity Score (Real): 0.5528 (Fake): 0.6388
Similarity Score (Real): 0.3662 (Fake): 0.2200
Similarity Score (Real): 0.6348 (Fake): 0.3492
Similarity Score (Real): 0.4255 (Fake): 0.3843
Similarity Score (Real): 0.3348 (Fake): 0.2929
Similarity Score (Real): 0.4901 (Fake): 0.6606
Similarity Score (Real): 0.3695 (Fake): 0.4951
Similarity Score (Real): 0.1753 (Fake): 0.1859
Similarity Score (Real): 0.4523 (Fake): 0.5537
Similarity Score (Real): 0.4228 (Fake): 0.5019
Similarity Score (Real): 0.2109 (Fake): 0.3261
Similarity Score (Real): 0.5251 (Fake): 0.4770
Similarity Score (Real): 0.2906 (Fake): 0.3335
Similarity Score (Real): 0.2952 (Fake): 0.1849
Similarity Score (Real): 0.5885 (Fake): 0.4812
Similarity Score (Real): 0.3511 (Fake): 0.3551
Similarity Score (Real): 0.2923 (Fake): 0.2174
Similarity Score (Real): 0.5661 (Fake): 0.5617
Similarity Score (Real): 0.3109 (Fake): 0.3501
Similarity Score (Real): 0.1555 (Fake): 0.1666
Similarity Sc

In [20]:
#Saving Models

In [22]:
torch.save(generator, 'generator_model.pth')
torch.save(discriminator, 'discriminator_model.pth')


In [ ]:
import os
import torch
import torch.nn as nn
import numpy as np
from torchvision.utils import save_image
import matplotlib.pyplot as plt  # Importing Matplotlib for plotting

# Create the directory for fake images if it doesn't exist
if not os.path.exists('./Fake_Images'):
    os.makedirs('./Fake_Images')

# Generator network
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100, 256, 4, 1, 0, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )
    
    def forward(self, x):
        return self.main(x)

# Siamese-like Discriminator network
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.fc = nn.Sequential(
            nn.Linear(256*4*4*2, 1),  # 2 because we concatenate two images (real + generated)
            nn.Sigmoid()
        )
    
    def forward(self, img_real, img_generated):
        features_real = self.cnn(img_real).view(img_real.size(0), -1)
        features_generated = self.cnn(img_generated).view(img_generated.size(0), -1)
        combined_features = torch.cat((features_real, features_generated), 1)
        similarity_score = self.fc(combined_features)
        
        return similarity_score

# Initialize networks
generator = Generator()
discriminator = Discriminator()

# Set the loss criterion
criterion = nn.BCELoss()

# Optimizers for Generator and Discriminator
optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Define number of epochs
epochs = 50

# Check if CUDA is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the models to the selected device
generator.to(device)
discriminator.to(device)

# Lists to hold the loss values
d_losses = []
g_losses = []

# Training loop
for epoch in range(epochs):
    for i, (imgs, _) in enumerate(train_loader):
        # Ground truths for real and fake
        real_labels = torch.ones(imgs.size(0), 1).to(device)
        fake_labels = torch.zeros(imgs.size(0), 1).to(device)
        
        # Move real images to device
        imgs = imgs.to(device)

        ### Train Discriminator ###
        noise = torch.randn(imgs.size(0), 100, 1, 1).to(device)
        fake_imgs = generator(noise)
        
        real_score = discriminator(imgs, imgs)  # Similarity score for real
        fake_score = discriminator(imgs, fake_imgs)  # Similarity score for fake
        
        d_loss_real = criterion(real_score, real_labels)
        d_loss_fake = criterion(fake_score, fake_labels)
        
        d_loss = d_loss_real + d_loss_fake
        
        optimizer_D.zero_grad()
        d_loss.backward()
        optimizer_D.step()

        ### Train Generator ###
        noise = torch.randn(imgs.size(0), 100, 1, 1).to(device)
        fake_imgs = generator(noise)
        
        fake_score = discriminator(imgs, fake_imgs)
        
        g_loss = criterion(fake_score, real_labels)
        
        optimizer_G.zero_grad()
        g_loss.backward()
        optimizer_G.step()

        # Save losses for plotting
        d_losses.append(d_loss.item())
        g_losses.append(g_loss.item())

        # Save generated fake images every few epochs
        if (i + 1) % 100 == 0:  # Save every 100 batches
            save_image(fake_imgs.data, f'./Fake_Images/fake_images_epoch_{epoch+1}_batch_{i+1}.png', normalize=True)

    print(f"Epoch [{epoch}/{epochs}] | D Loss: {d_loss.item():.4f} | G Loss: {g_loss.item():.4f}")

# Print last recorded losses
print(f"Final D Loss: {d_losses[-1]:.4f} | Final G Loss: {g_losses[-1]:.4f}")

# Plotting the losses
plt.figure(figsize=(10, 5))
plt.plot(d_losses, label='Discriminator Loss', color='red')
plt.plot(g_losses, label='Generator Loss', color='blue')
plt.title('Losses during Training')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.legend()
plt.show()
